In [29]:
import random
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm
import statsmodels.stats.multitest as smt

In [42]:
def sample_gen(n ) :
    arr = []
    for j in range(n) :
        arr.append([np.random.normal(0,1.0, n) , np.random.normal(0,1.0)])
    return np.array(arr).T
def sample_gen(n ) :
    # print(np.array( list(zip(np.random.normal(0,1.0, n) , np.random.normal(0,1.0, n)))).T )
    return np.array( list(zip(np.random.normal(0,1.0, n) , np.random.normal(0,1.0, n)))).T

def get_corr_sample(sample , rho) :
    alpha = (np.sqrt(1+rho) + np.sqrt(1-rho))/2
    beta = (np.sqrt(1+rho) - np.sqrt(1-rho))/2
    A_rho = [[alpha , beta],[beta , alpha]]
    return A_rho@sample
def T(X) :
    return np.sum(X[0]*X[1])/len(X[0])
def T_2(X) :
    return np.sum((X[0]-X[1])**2)/len(X[0])/2
def get_W(n , criterion , num_samples ,rho_arr) :
    W = []
    for rho in rho_arr :
        count_reject = []
        for _ in range(num_samples) :
            sample = sample_gen(n)
            sample = get_corr_sample(sample , rho)
            count_reject.append(criterion(sample))
        W.append( sum(count_reject) / num_samples)
    return W
def crit_1(sample)  :
    return np.abs(T(sample)) > 20/n
def crit_2(sample) :
    return np.abs(T_2(sample)-1) > 1.96*np.sqrt(2)/np.sqrt(n)
def crit_1_by_alpha(sample , alpha ) :
    return np.abs(T(sample)) > 1/(len(sample) * alpha)

In [38]:
def get_p_values(num_samples , n=100 ) :
    p_values = []
    sigma = np.sqrt(1/n)
    is_zero_rho = []
    sample = sample_gen(n)
    sample = get_corr_sample(sample , 0 )
    stat = np.abs(T(sample))
    left = norm.cdf( -stat , loc=0, scale=sigma )
    p_values.append(2*left)
    is_zero_rho.append(0)
    rho = 0
    for i in range(1,num_samples) :
        z = np.random.binomial(1, 0.3, size=1)
        if z :
            rho = rho
        else :
            rho = 0.2 - rho
        sample = sample_gen(n)
        sample = get_corr_sample(sample , rho)
        stat = np.abs(T(sample))
        left = norm.cdf( -stat , loc=0, scale=sigma )
        p_values.append(2*left)
        if rho == 0 :
            is_zero_rho.append(i)
    return np.array(p_values)  , np.array(is_zero_rho)

In [32]:
n = 100
num_samples = 1000


FWER без поправок

In [15]:
N_sampling = 100
counter = 0
for j in range(N_sampling) :
    p_value , is_zero = get_p_values(num_samples,n)
    counter += np.sum(p_value[is_zero] < 0.05 ) > 0 

print(counter /N_sampling)

1.0


FWER  с поправкой

In [36]:
N_sampling = 100
counter = 0
for j in range(N_sampling) :
    p_value , is_zero = get_p_values(num_samples,n)
    new_p_value = p_value[is_zero]*1000
    counter += np.sum(new_p_value< 0.05 ) > 0

print(counter /N_sampling) 

0.03


FDR без поправки

In [17]:
N_sampling = 100
value = 0
for j in range(N_sampling) :
    p_value , is_zero = get_p_values(num_samples,n)
    
    cnt_1 = np.sum( p_value[is_zero] < 0.05 )
    cnt_2 = np.sum( p_value < 0.05)
    if cnt_2 != 0 :
        value += cnt_1/cnt_2

print( value/N_sampling) 

0.08974673874788236


#### FDR с поправкой

In [64]:
N_sampling = 100
value = 0
for j in range(N_sampling) :
    p_value , is_zero = get_p_values(num_samples,n)

    sorted_index = np.argsort(p_value)
    p_value = p_value[sorted_index]
    corrected_p_value = smt.multipletests( p_value, method='fdr_bh' ,is_sorted =True)[1]
    inverse_index = np.argsort(sorted_index)
    new_p_value = corrected_p_value[inverse_index]

    cnt_1 = np.sum( new_p_value[is_zero] < 0.05 )
    cnt_2 = np.sum( new_p_value < 0.05)
    if cnt_2 != 0 :
        value += cnt_1/cnt_2

print( value/N_sampling) 

0.026206268358793996
